In [3]:
from facemesh.faceDetector import FaceDetector, cv2
import numpy as np
import time

def draw_polyline(img, l):
    cv2.polylines(img, np.array([l], dtype=np.int32), True, (0,255,0), 1, cv2.LINE_AA)

## Landmarks keypoints:
## https://github.com/tensorflow/tfjs-models/blob/838611c02f51159afdd77469ce67f0e26b7bbb23/face-landmarks-detection/src/mediapipe-facemesh/keypoints.ts
lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 
LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

cap = cv2.VideoCapture(0)
pTime = 0
detector = FaceDetector()


while True:
    left_eye_lm = []
    right_eye_lm = []
    lipsUpperOuter_lm, lipsLowerOuter_lm, lipsUpperInner_lm, lipsLowerInner_lm = [],[],[],[]
    success, img = cap.read()
    if success:
        l = detector.facemeshing(img, draw = False)
        if l:
            print("face detected")
            # Get left and right eye
            for lm_id in LEFT_EYE:
                left_eye_lm.append(l[0][lm_id])
            
            for lm_id in RIGHT_EYE:
                right_eye_lm.append(l[0][lm_id])
            # Get lips: 
            for lm_id in lipsUpperOuter:
                lipsUpperOuter_lm.append(l[0][lm_id])
            
            for lm_id in lipsLowerOuter:
                lipsLowerOuter_lm.append(l[0][lm_id])
            for lm_id in lipsUpperInner:
                lipsUpperInner_lm.append(l[0][lm_id])
            
            for lm_id in lipsLowerInner:
                lipsLowerInner_lm.append(l[0][lm_id])
            
            draw_polyline(img,left_eye_lm )
            draw_polyline(img,right_eye_lm )
            draw_polyline(img,lipsUpperOuter_lm + lipsLowerOuter_lm[::-1])
            draw_polyline(img,lipsUpperInner_lm + lipsLowerInner_lm[::-1])
        else:
            print("no face detected.")
    cTime = time.time()
    fps = 1 / (cTime-pTime)
    pTime = cTime
    
    cv2.putText(img, f'FPS: {int(fps)}', (20,50), cv2.FONT_HERSHEY_PLAIN,3, (0,255, 255), 2)
    cv2.imshow('face',img)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release() 
        break  
cv2.destroyAllWindows()



face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected
face detected


In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np

class FaceDetector():
    def __init__(self, minDetConf = 0.5):
        self.mpFaceDetection = mp.solutions.face_detection
        self.mpDraw = mp.solutions.drawing_utils
        self.faceDetection = self.mpFaceDetection.FaceDetection(minDetConf)
        self.mpFaceMesh = mp.solutions.face_mesh
        self.mpFaceMeshConnections = mp.solutions.face_mesh_connections
        self.faceMesh = self.mpFaceMesh.FaceMesh(
            max_num_faces=2,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5)
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1, circle_radius=1)
    
    def findFaces(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceDetection.process(imgRGB)
        bboxs = []
        if results.detections:
            for face_id, detection in enumerate(results.detections):
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, ic = img.shape
                bbox =  int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                        int(bboxC.width * iw), int(bboxC.height * ih)
                if draw: 
                    self.squareDraw(img, bbox)
                    cv2.putText(img, 
                        f'{ int(detection.score[0]*100) }%', 
                        (bbox[0], bbox[1]-20),
                        cv2.FONT_HERSHEY_PLAIN, 
                        2, (255, 0, 255), 2)
                bboxs.append([bbox, detection.score])
        return bboxs

    def squareDraw(self, img, bbox, l = 20, refctangle_t = 1, corner_t = 5):
        x, y, w, h = bbox #first point, bridging point
        x1, y1 = x+w, y+h #bottom right point
        corner_color_y =  (0,225,128)
        corner_color_x =  (0,225,128)
        rectangle_color = (255,0,255)

        cv2.rectangle(img, bbox, rectangle_color, refctangle_t)
        cv2.line(img, (x,y), (x+l,y), corner_color_x, corner_t )
        cv2.line(img, (x,y), (x,y+l), corner_color_y, corner_t )

        cv2.line(img, (x1,y), (x1-l,y), corner_color_x, corner_t )
        cv2.line(img, (x1,y), (x1,y+l), corner_color_y, corner_t )

        cv2.line(img, (x,y1), (x+l,y1), corner_color_x, corner_t )
        cv2.line(img, (x,y1), (x,y1-l), corner_color_y, corner_t )

        cv2.line(img, (x1,y1), (x1-l,y1), corner_color_x, corner_t )
        cv2.line(img, (x1,y1), (x1,y1-l), corner_color_y, corner_t )
    
    def facemeshing(self, img, draw = True, draw_square = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(imgRGB)
        faces = []
        face = {}
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, faceLms, self.mpFaceMesh.FACEMESH_LEFT_EYE,
                                            self.drawSpec,self.drawSpec )

                for id, lm in enumerate(faceLms.landmark):
                    ih, iw, ic = img.shape
                    x,y = int(lm.x*iw), int(lm.y*ih)
                    #faces.append({id: [x,y]}) 
                    face[id]= [x,y]     ## adds every 478 landmark with its coordetates, for each face in list
                faces.append(face)
            return faces ## 1 or 2 faces in list